<a href="https://colab.research.google.com/github/Anyaoma/My-projects/blob/main/Backtest_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from ClassOanda import OandaTrader1

In [2]:
class BacktestBase:
    def __init__(self, symbol, start, end, amount, ftc= 0.0, ptc=0.0, verbose=True):
        self.position = 0
        self.trade = 0
        self.units = 0
        self.start = start
        self.end = end
        self._initial_amount = amount
        self.amount = self._initial_amount
        self.ptc = ptc
        self.ftc = ftc
        self.verbose = verbose
        self.symbol = symbol
        self.get_data()

    def get_data(self):
        data = pd.read_csv('http://hilpisch.com/pyalgo_eikon_eod_data.csv',index_col=0, parse_dates=True).dropna()
        data = pd.DataFrame(data[self.symbol])
        data.rename(columns={self.symbol: 'price'}, inplace=True)
        data['return'] = np.log(data/data.shift(1))
        self.data = data.dropna()

    def plot_data(self,cols=None):
        if cols is None:
            cols = ['price']
        self.data[cols].plot(figsize=(10,6), title=self.symbol)

    def get_date_and_price(self,bar):
        date = str(self.data.index[bar])[:10]
        price = self.data['price'].iloc[bar]
        return date, price

    def print_balance(self, bar):
        date, price = self.get_date_and_price(bar)
        print(f'{date} | Current balance is {self.amount:.2f}')

    def print_net_wealth(self, bar):
        date, price = self.get_date_and_price(bar)
        net_wealth = self.units * price + self.amount
        print(f'{date} | Total wealth is: {net_wealth}')

    def place_buy_trade(self, bar, units=None, amount=None):
        date, price = self.get_date_and_price(bar)
        if units is None:
            units = amount/price
        self.amount -= units*price * (1 + self.ptc) + self.ftc
        self.units += units
        self.trade += 1
        if self.verbose:
            print(f'{date} | buying {units} units at {price:.2f}')
            self.print_balance(bar)
            self.print_net_wealth(bar)

    def place_sell_trade(self, bar, units=None, amount=None):
        date, price = self.get_date_and_price(bar)
        if units is None:
            units = amount/price
        self.amount += units*price * (1- self.ptc) - self.ftc
        self.units -= units
        self.trade += 1
        if self.verbose:
            print(f'{date} | selling {units} units at {price:.2f}')
            self.print_balance(bar)
            self.print_net_wealth(bar)

    def close_out(self, bar):
        date, price = self.get_date_and_price(bar)
        self.amount += self.units * price
        self.units = 0
        self.trade += 1
        if self.verbose:
            print(f'{date} | inventory {self.units} units at{price:.2f}')
            print('=' * 55)
        print('Final balance [$] {:.2f}'.format(self.amount))
        perf = ((self.amount - self._initial_amount) / self._initial_amount * 100)
        print('Net Performance [%] {:.2f}'.format(perf))
        print('Trades Executed [#] {:.2f}'.format(self.trade))
        print('=' * 55)







# Long-Only Backtesting Class

In [3]:
class LongOnlyBacktest(BacktestBase):
    def __init__(self, symbol, start, end, amount,ftc= 0.0, ptc=0.0, verbose=True):
        super().__init__(symbol, start, end, amount,ftc= 0.0, ptc=0.0, verbose=True)


    def run_mean_reversion_strategy(self, SMA, threshold):
        ''' Backtesting a mean reversion-based strategy.
            Parameters
            ==========
            SMA: int
            simple moving average in days
            threshold: float
            absolute value for deviation-based signal relative to SMA
        '''
        msg = f'\n\nRunning mean reversion strategy | '
        msg += f'SMA={SMA} & thr={threshold}'
        msg += f'proportional cost {self.ptc}'
        print(msg)
        print('='*55)
        self.position = 0
        self.trade = 0
        self.amount = self._initial_amount

        self.data['SMA'] = self.data['price'].rolling(SMA).mean()
        for bar in range(SMA, len(self.data)):
            if self.position == 0:
                if (self.data.price.iloc[bar] < self.data.SMA.iloc[bar] - threshold):
                    self.place_buy_trade(bar, amount=self.amount)
                    self.position = 1
            if self.position == 1:
                if (self.data.price.iloc[bar] > self.data.SMA.iloc[bar]):
                    self.place_sell_trade(bar, units=self.units)
                    self.position = 0
        self.close_out(bar)

    def run_sma_strategy(self, SMA1, SMA2):
        ''' Backtesting an SMA-based strategy.
            Parameters
            ==========
            SMA1, SMA2: int
            shorter and longer term simple moving average (in days)
        '''
        assert int(SMA2) > int(SMA1)
        msg = f'\n\nRunning SMA strategy | SMA1={SMA1} & SMA2={SMA2}'
        msg += f'\nfixed costs {self.ftc} | '
        msg += f'proportional costs {self.ptc}'
        print(msg)
        print('=' * 55)
        self.position = 0
        self.trades = 0
        self.amount = self._initial_amount

        self.data['SMA1'] = self.data['price'].rolling(SMA1).mean()
        self.data['SMA2'] = self.data['price'].rolling(SMA2).mean()

        for bar in range(SMA2, len(self.data)):
            if self.position == 0:
                if self.data.SMA1.iloc[bar] > self.data.SMA2.iloc[bar]:
                    self.place_buy_trade(bar, amount=self.amount)
                    self.position = 1
            elif self.position == 1:
                if self.data.SMA1.iloc[bar] < self.data.SMA2.iloc[bar]:
                    self.place_sell_trade(bar, units=self.units)
                    self.position = 0
        self.close_out(bar)


# Long-Short Backtesting Class

In [4]:
class LongShortBacktest(BacktestBase):
    def __init__(self, symbol, start, end, amount,ftc= 0.0, ptc=0.0, verbose=True):
        super().__init__(symbol, start, end, amount,ftc= 0.0, ptc=0.0, verbose=True)


    def go_long(self, bar, units=None, amount=None):
        if self.position == -1:
            self.place_buy_trade(bar, units=-self.units)
        if units:
            self.place_buy_trade(bar, units)
        elif amount:
            if amount == 'all':
                amount = self.amount
            self.place_buy_trade(bar, amount=amount)

    def go_short(self, bar, units=None, amount=None):
        if self.position == 1:
            self.place_sell_trade(bar, units=self.units)
        if units:
            self.place_sell_trade(bar, units=units)
        elif amount:
            if amount == 'all':
                amount = self.amount
            self.place_sell_trade(bar, amount=amount)


    def run_sma_strategy(self, SMA1, SMA2):
        msg = f'\n\nRunning SMA strategy | SMA1={SMA1} & SMA2={SMA2}'
        msg += f'\nfixed costs {self.ftc} | '
        msg += f'proportional costs {self.ptc}'
        print(msg)
        print('=' * 55)

        self.position = 0
        self.trade = 0
        self.amount = self._initial_amount

        self.data['SMA1'] = self.data['price'].rolling(SMA1).mean()
        self.data['SMA2'] = self.data['price'].rolling(SMA2).mean()

        for bar in range(SMA2, len(self.data)):
            if self.position in [0, -1]:
                if self.data['SMA1'].iloc[bar] > self.data['SMA2'].iloc[bar]:
                    self.go_long(bar, amount='all')
                    self.position = 1
            if self.position in [0,1]:
                if self.data['SMA1'].iloc[bar] < self.data['SMA2'].iloc[bar]:
                    self.go_short(bar, amount='all')
                    self.position = -1
        self.close_out(bar)

    def run_mean_reversion_strategy(self, SMA, threshold):
        msg = f'\n\nRunning mean reversion strategy | '
        msg += f'SMA={SMA} & thr={threshold}'
        msg += f'\nfixed costs {self.ftc} | '
        msg += f'proportional costs {self.ptc}'
        print(msg)
        print('=' * 55)
        self.position = 0
        self.trade = 0
        self.amount = self._initial_amount

        self.data['SMA'] = self.data['price'].rolling(SMA).mean()

        for bar in range(SMA, len(self.data)):
            if self.position == 0:
                if (self.data['price'].iloc[bar] < self.data['SMA'].iloc[bar] - threshold):
                    self.go_long(bar, amount=self.amount)
                    self.position = 1
                elif (self.data['price'].iloc[bar] > self.data['SMA'].iloc[bar] + threshold):
                    self.go_short(bar, amount = self.amount)
                    self.position = -1
            elif self.position == 1:
                if self.data['price'].iloc[bar] >= self.data['SMA'].iloc[bar]:
                    self.place_sell_trade(bar, units=self.units)
                    self.position = 0
            elif self.position == -1:
                if self.data['price'].iloc[bar] <= self.data['SMA'].iloc[bar]:
                    self.place_buy_trade(bar, units=-self.units)
                    self.position = 0
        self.close_out(bar)





![image.png](attachment:7e997d22-d29f-4d1c-b908-3414e4bc969e.png)

![image.png](attachment:cc4d0a16-21eb-4143-870f-be66598b79fe.png)

In [5]:
def run_strategies():
    lsbt.run_sma_strategy(42, 252)
    lsbt.run_mean_reversion_strategy(50,5)

lsbt = LongShortBacktest('EUR=', '2010-1-1', '2019-12-31', 10000,verbose=False)
run_strategies()



Running SMA strategy | SMA1=42 & SMA2=252
fixed costs 0.0 | proportional costs 0.0
2011-01-04 | buying 7518.2317119013605 units at 1.33
2011-01-04 | Current balance is 0.00
2011-01-04 | Total wealth is: 10000.0
2011-10-07 | selling 7518.2317119013605 units at 1.34
2011-10-07 | Current balance is 10055.63
2011-10-07 | Total wealth is: 10055.63491466807
2011-10-07 | selling 7518.2317119013605 units at 1.34
2011-10-07 | Current balance is 20111.27
2011-10-07 | Total wealth is: 10055.63491466807
2012-11-01 | buying 7518.2317119013605 units at 1.29
2012-11-01 | Current balance is 10381.17
2012-11-01 | Total wealth is: 10381.174347793398
2012-11-01 | buying 8021.306094725234 units at 1.29
2012-11-01 | Current balance is 0.00
2012-11-01 | Total wealth is: 10381.174347793398
2014-07-22 | selling 8021.306094725234 units at 1.35
2014-07-22 | Current balance is 10799.89
2014-07-22 | Total wealth is: 10799.886525938056
2014-07-22 | selling 8021.306094725234 units at 1.35
2014-07-22 | Current bal